In [1]:
import minitorch as mt
from minitorch import nn,optim,functional as F

In [2]:
import numpy as np
import torch
from torchvision import datasets, transforms

In [3]:
BATCH_SIZE=128
EPOCHS=5 # 总共训练批次

In [4]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self): 
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2 = nn.Conv2d(10,20,3) # 128, 10x10
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
        
    
    def forward(self,x:mt.Tensor):
        # x： 512-1-28-28
        batch_size = x.shape[0]
        out = self.conv1(x)     # 512-10-24-24
        out = F.relu(out)
        out = F.maxpool2d(out,(2,2)) # 512-10-12-12
        out = self.conv2(out) # 512-20-10-10
        out = F.relu(out)
        out = out.reshape((batch_size,-1))
        out = F.dropout(out)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return F.log_softmax(out,dim=1)

In [6]:

def train(model, train_loader, optimizer, epoch):
    model.switch_train_mode()
    for batch_idx, (data, target) in enumerate(train_loader):
        data,target = mt.from_numpy(data.numpy().astype(np.float64)),target.numpy()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step(epoch)
        if(batch_idx+1)%40 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model,test_loader):
    test_loss = 0
    correct = 0
    model.switch_eval_mode()
    for data, target in test_loader:
        data,target = mt.from_numpy(data.numpy().astype(np.float64)),target.numpy()
        output = model(data)
        test_loss += F.nll_loss(output, target).item() # 将一批的损失相加
        result = output.data.argmax(axis=1)
        correct += (result == target).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = ConvNet()
model

ConvNet(
   (conv1): Conv2d(in_channel=1,out_channel=10,kernel_size=5,bias=True)
   (conv2): Conv2d(in_channel=10,out_channel=20,kernel_size=3,bias=True)
   (dropout): Dropout(p=0.1)
   (fc1): Linear(in_features=2000, out_features=500,bias=True)
   (fc2): Linear(in_features=500, out_features=10,bias=True)
)

In [8]:
%%time
EPOCHS = 1
model = ConvNet()
#测试１：
# ConvNet_1_2() batch_size = 128  可达到 99%
# lr = [(3,0.05),(5,0.02),(7,0.001),(10,0.005),(15,0.001),(20,0.0005)]
# optimizer = optim.SGD(model.parameters(),lr,momentum=0.9)

# 测试2
# lr = [(3,0.05),(5,0.02),(7,0.001),(10,0.005),(15,0.001),(20,0.0005)]
# optimizer = optim.Adagrad(model.parameters(),lr=lr) 

# 128, 5 epoch 可达到99%
# optimizer = optim.Adadelta(model.parameters())

# 测试失败
# optimizer = optim.RMSprop(model.parameters(),beta=0.5)

# 128, 20次可达到99.30%
optimizer = optim.Adam(model.parameters())

for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test(model,test_loader)

Train Epoch: 1 [4992/60000 (8%)]	Loss: 0.192004
Train Epoch: 1 [10112/60000 (17%)]	Loss: 0.212411
Train Epoch: 1 [15232/60000 (25%)]	Loss: 0.172954
Train Epoch: 1 [20352/60000 (34%)]	Loss: 0.231436
Train Epoch: 1 [25472/60000 (42%)]	Loss: 0.160820
Train Epoch: 1 [30592/60000 (51%)]	Loss: 0.087430
Train Epoch: 1 [35712/60000 (59%)]	Loss: 0.105173
Train Epoch: 1 [40832/60000 (68%)]	Loss: 0.143400
Train Epoch: 1 [45952/60000 (77%)]	Loss: 0.114819
Train Epoch: 1 [51072/60000 (85%)]	Loss: 0.064155
Train Epoch: 1 [56192/60000 (94%)]	Loss: 0.083155

Test set: Average loss: 0.0007, Accuracy: 9717/10000 (97.17%)

CPU times: user 6min 35s, sys: 3min 35s, total: 10min 10s
Wall time: 1min 32s


In [9]:
model

ConvNet(
   (conv1): Conv2d(in_channel=1,out_channel=10,kernel_size=5,bias=True)
   (conv2): Conv2d(in_channel=10,out_channel=20,kernel_size=3,bias=True)
   (dropout): Dropout(p=0.1)
   (fc1): Linear(in_features=2000, out_features=500,bias=True)
   (fc2): Linear(in_features=500, out_features=10,bias=True)
)

In [18]:
model.conv1.weight = 0

In [20]:
model._layers['conv1'].weight

0